In [1]:
import numpy as np
from hsmm_base import GaussianHSMM

In [2]:
# sample observations (from hsmmlearn)
obs = np.array([9.7471989, 9.01877392, -1.28428112, 0.55009463, 0.58425178, 0.43354177,
                10.63374352, 10.14025491, 5.3215164, 5.46421001, 5.16719792, 9.12216889,
                10.49171453, -0.72812025, 0.57309517, 0.3420868, -1.35338431, 4.12587557,
                6.907117, 5.41243634])

In [3]:
R = GaussianHSMM(n_states = 3, n_durations = 4)

In [4]:
# initial parameters (from hsmmlearn tutorial)

R.pi = np.array([1 / 3, 1 / 3, 1 / 3])

R.dur = np.array([
    [0.1, 0.005, 0.005, 0.89],
    [0.1, 0.005, 0.89, 0.005],
    [0.1, 0.89, 0.005, 0.005]
])

R.tmat = np.array([
        [0.0, 0.5, 0.5],
        [0.3, 0.0, 0.7],
        [0.6, 0.4, 0.0]
    ])

R.mean = np.array([0.0, 5.0, 10.0])
R.sdev = np.array([1, 1, 1])

In [5]:
# EM algorithm
R.fit(obs, censoring = 1)

FIT: reestimation complete for 1th loop.
FIT: reestimation complete for 2th loop.
FIT: converged at 3th loop.


In [6]:
# Log-probability of time-series under the model
R.score(obs, censoring = 1)

-25.87422389268413

In [7]:
# new parameters
print("Start Probabilities:\n", R.pi, "\n")
print("Transition Matrix:\n", R.tmat, "\n")
print("Durations:\n", R.dur, "\n")
print("Means:\n", R.mean, "\n")
print("Std. Deviations:\n", R.sdev)

Start Probabilities:
 [1.08688135e-69 2.37979076e-22 1.00000000e+00] 

Transition Matrix:
 [[0.00000000e+00 5.00000000e-01 5.00000000e-01]
 [2.56937871e-19 0.00000000e+00 1.00000000e+00]
 [6.66666667e-01 3.33333333e-01 0.00000000e+00]] 

Durations:
 [[5.07959969e-27 8.67694653e-33 2.74135825e-21 1.00000000e+00]
 [8.41014588e-14 3.83344255e-19 1.00000000e+00 1.10263279e-16]
 [5.61360746e-14 1.00000000e+00 3.12656940e-29 8.27053039e-40]] 

Means:
 [-0.11033944  5.39972554  9.85897578] 

Std. Deviations:
 [0.80554035 0.8123452  0.62464109]


In [8]:
# Viterbi algorithm
states, logprob = R.predict(obs, censoring = 1)

In [9]:
# correct states (from hsmmlearn tutorial)
correct = np.array([2,2,0,0,0,0,2,2,1,1,1,2,2,0,0,0,0,1,1,1])

In [10]:
# how many incorrect states?
np.sum(states != correct)

0